In [1]:
# -*- coding: utf-8 -*-
"""
YOLOv8 Training & Evaluation on Roboflow Dataset (Version 5)

This notebook trains a YOLOv8 model on a specific version of the
'vehicle-detection-2-my3ke' dataset from Roboflow and reports
detailed performance metrics.
"""

# ----------------------------------------
# Step 1: Environment Setup
# ----------------------------------------
print("Step 1: Setting up the environment...")

# Install necessary libraries
# We install ultralytics (which includes YOLOv8) and roboflow
!pip install ultralytics roboflow -q

# Import required libraries
import os
import yaml
import torch
from ultralytics import YOLO
from roboflow import Roboflow
import ultralytics # To check version

# Verify ultralytics version (should be 8.x.x)
print(f"Ultralytics Version: {ultralytics.__version__}")
# Check if GPU is available
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Current CUDA device: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("WARNING: GPU not found, training will be significantly slower.")

print("Environment setup complete.")
print("-" * 30)

# ----------------------------------------
# Step 2: Dataset Download
# ----------------------------------------
print("Step 2: Downloading dataset from Roboflow...")

# --- IMPORTANT ---
# Replace 'YOUR_API_KEY' with your actual Roboflow API Key
# Note: You provided a key, but it's best practice not to hardcode keys directly.
# For this example, I'll use the key you provided, but be mindful of security.
API_KEY = "mkCcH6GnK5SrMsAic3m6" # Use the key you provided

try:
    rf = Roboflow(api_key=API_KEY)
    project = rf.workspace("smartflow-2xtbl").project("vehicle-detection-2-my3ke")
    version = project.version(5) # Using version 5 as requested
    dataset = version.download("yolov8") # Download in YOLOv8 format

    # Get dataset location and data.yaml path
    dataset_location = dataset.location
    data_yaml_path = os.path.join(dataset_location, 'data.yaml')

    print(f"Dataset downloaded successfully to: {dataset_location}")
    print(f"Data configuration file: {data_yaml_path}")

    # Optional: Display data.yaml contents to verify classes
    print("\n--- data.yaml Contents ---")
    with open(data_yaml_path, 'r') as f:
        data_yaml_content = yaml.safe_load(f)
        print(yaml.dump(data_yaml_content, default_flow_style=False))
    print("--- End data.yaml ---")

except Exception as e:
    print(f"Error downloading dataset: {e}")
    print("Please ensure your API key is correct and has access to the project.")
    # Stop execution if dataset download fails
    raise SystemExit("Dataset download failed.")

print("Dataset download complete.")
print("-" * 30)

# ----------------------------------------
# Step 3: Define Hyperparameters
# ----------------------------------------
print("Step 3: Defining hyperparameters...")

EPOCHS = 50
PATIENCE = 20
IMGSZ = 640
BATCH = 16
PLOTS = True # Generate plots during training/validation
SAVE = True  # Save checkpoints and final model

print(f"Epochs: {EPOCHS}")
print(f"Patience: {PATIENCE}")
print(f"Image Size: {IMGSZ}")
print(f"Batch Size: {BATCH}")
print(f"Generate Plots: {PLOTS}")
print(f"Save Model: {SAVE}")
print("Hyperparameters defined.")
print("-" * 30)

# ----------------------------------------
# Step 4: Model Training
# ----------------------------------------
print("Step 4: Starting YOLOv8 model training...")

# Load a pre-trained YOLOv8 model (e.g., yolov8n.pt for nano, yolov8s.pt for small)
# Using yolov8n.pt as a common starting point. Change if needed.
model = YOLO('yolov8s.pt')

# Define a unique name for this training run
run_name = f'yolov8n_vehicle_v5_e{EPOCHS}_b{BATCH}'

try:
    # Start training
    results = model.train(
        data=data_yaml_path,
        epochs=EPOCHS,
        patience=PATIENCE,
        imgsz=IMGSZ,
        batch=BATCH,
        plots=PLOTS,
        save=SAVE,
        name=run_name, # Name the output directory
        val=True # Ensure validation is performed during training
    )
    print("Training finished.")
    # The best model weights are automatically saved (e.g., runs/detect/YOUR_RUN_NAME/weights/best.pt)
    best_model_path = os.path.join('runs', 'detect', run_name, 'weights', 'best.pt')
    print(f"Best model saved at: {best_model_path}")

except Exception as e:
    print(f"Error during training: {e}")
    raise SystemExit("Training failed.")

print("Model training complete.")
print("-" * 30)


# ----------------------------------------
# Step 5: Model Evaluation and Metrics Reporting
# ----------------------------------------
print("Step 5: Evaluating the best model and reporting metrics...")

if 'best_model_path' in locals() and os.path.exists(best_model_path):
    print(f"Loading best model from: {best_model_path}")
    # Load the best saved model
    best_model = YOLO(best_model_path)

    # Evaluate the model on the validation set
    print("Running validation on the 'val' split...")
    metrics = best_model.val(
        data=data_yaml_path,
        imgsz=IMGSZ,
        batch=BATCH,
        split='val', # Explicitly use the validation split
        plots=True,  # Generate validation plots (like confusion matrix)
        save_json=True # Save results to JSON, includes COCO metrics
    )

    print("\n--- Overall Dataset Metrics ---")
    # Box metrics are accessed via metrics.box
    map50_95 = metrics.box.map    # mAP50-95
    map50 = metrics.box.map50   # mAP50
    # Note: metrics.box.p and metrics.box.r usually store the overall P and R at the optimal confidence threshold
    # Often, the most representative P and R are those associated with mAP50 calculation or F1-score maximization.
    # Let's report the mean precision and recall across classes as reported by the summary.
    # The printed output summary usually shows these clearly. We extract them from the object here:
    # The `results_dict` contains metrics keyed by strings.
    overall_precision = metrics.results_dict.get('metrics/precision(B)', 'N/A')
    overall_recall = metrics.results_dict.get('metrics/recall(B)', 'N/A')

    print(f"Box Precision (P): {overall_precision:.4f}")
    print(f"Box Recall (R):    {overall_recall:.4f}")
    print(f"Box mAP50:         {map50:.4f}")
    print(f"Box mAP50-95:      {map50_95:.4f}")


    print("\n--- Per-Class Metrics ---")
    class_names = metrics.names # Dictionary mapping class index to class name {0: 'classA', 1: 'classB', ...}
    num_classes = len(class_names)

    # Get per-class AP50 and AP50-95 values
    # metrics.box.maps contains mAP50-95 per class (index corresponds to class index)
    per_class_map50_95 = metrics.box.maps

    # To get per-class P, R, and mAP50, we need to look deeper or parse the standard output table structure.
    # The `metrics` object doesn't always expose these directly per class in a simple array format.
    # Let's try to access them if available, otherwise state 'N/A' or parse from results.
    # Often, the confusion matrix can give insights, or specific result files generated.
    # For now, let's focus on mAP50 and mAP50-95 per class which are readily available.
    # We can *infer* P and R per class from the confusion matrix plots generated if needed,
    # but direct numeric access isn't guaranteed in a simple attribute for all versions/tasks.
    # The standard `val()` output usually prints a table with P, R, mAP50 per class.

    # We'll print mAP50-95 per class from `metrics.box.maps`
    # And acknowledge that P, R, mAP50 per class are best viewed in the printed table/plots
    # from the `val()` output or `results.png` / `confusion_matrix.png`.

    print("Class Name             | mAP50-95") # Add P, R, mAP50 if easily extractable later
    print("-----------------------|-----------")
    if per_class_map50_95 is not None and len(per_class_map50_95) == num_classes:
         for i in range(num_classes):
            class_name = class_names[i]
            class_map50_95 = per_class_map50_95[i]
            print(f"{class_name:<22} | {class_map50_95:.4f}")
    else:
         print("Per-class mAP50-95 data not readily available in the expected format.")

    print("\nNote: Per-class Precision (P), Recall (R), and mAP50 are typically displayed in the console output table during validation.")
    print("You can also find detailed plots (like confusion matrix, PR curve) in the run directory:")
    print(f"-> {os.path.join('runs', 'detect', run_name)}")

else:
    print("Evaluation could not be performed. Best model path not found or training failed.")

print("-" * 30)
print("Script finished.")

Step 1: Setting up the environment...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━


Extracting Dataset Version Zip to vehicle-detection-2-5 in yolov8:: 100%|██████████| 6236/6236 [00:01<00:00, 5778.88it/s]

Dataset downloaded successfully to: /content/vehicle-detection-2-5
Data configuration file: /content/vehicle-detection-2-5/data.yaml

--- data.yaml Contents ---
names:
- Ambulance
- Fire Truck
- regular vehicle
nc: 3
roboflow:
  license: Public Domain
  project: vehicle-detection-2-my3ke
  url: https://universe.roboflow.com/smartflow-2xtbl/vehicle-detection-2-my3ke/dataset/5
  version: 5
  workspace: smartflow-2xtbl
test: ../test/images
train: ../train/images
val: ../valid/images

--- End data.yaml ---
Dataset download complete.
------------------------------
Step 3: Defining hyperparameters...
Epochs: 50
Patience: 20
Image Size: 640
Batch Size: 16
Generate Plots: True
Save Model: True
Hyperparameters defined.
------------------------------
Step 4: Starting YOLOv8 model training...



100%|██████████| 21.5M/21.5M [00:00<00:00, 434MB/s]

Ultralytics 8.3.120 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


...

                   all        622       3120      0.913      0.861      0.931      0.729
             Ambulance        106        116      0.967      0.871      0.953      0.804
            Fire Truck        122        155      0.822      0.804      0.875      0.716
       regular vehicle        394       2849       0.95      0.908      0.965      0.667
Speed: 0.2ms preprocess, 4.4ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/yolov8n_vehicle_v5_e50_b16
Training finished.
Best model saved at: runs/detect/yolov8n_vehicle_v5_e50_b16/weights/best.pt
Model training complete.
------------------------------
Step 5: Evaluating the best model and reporting metrics...
Loading best model from: runs/detect/yolov8n_vehicle_v5_e50_b16/weights/best.pt
Running validation on the 'val' split...
Ultralytics 8.3.120 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs
val: Fast 

val: Scanning /content/vehicle-detection-2-5/valid/labels.cache... 622 images, 0 backgrounds, 0 corrupt: 100%|██████████| 622/622 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 6, len(boxes) = 3120. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [00:10<00:00,  3.71it/s]


                   all        622       3120      0.913       0.86      0.931      0.729
             Ambulance        106        116      0.967      0.871      0.953      0.806
            Fire Truck        122        155      0.822      0.804      0.876      0.716
       regular vehicle        394       2849      0.951      0.905      0.965      0.667
Speed: 0.8ms preprocess, 8.8ms inference, 0.0ms loss, 1.6ms postprocess per image
Saving runs/detect/val/predictions.json...
Results saved to runs/detect/val

--- Overall Dataset Metrics ---
Box Precision (P): 0.9134
Box Recall (R):    0.8601
Box mAP50:         0.9314
Box mAP50-95:      0.7295

--- Per-Class Metrics ---
Class Name             | mAP50-95
-----------------------|-----------
Ambulance              | 0.8056
Fire Truck             | 0.7160
regular vehicle        | 0.6667

Note: Per-class Precision (P), Recall (R), and mAP50 are typically displayed in the console output table during validation.
You can also find detailed plot